#### Prerequisite Libraries

In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.8 MB/s eta 0:00:00


#### Code and Output


In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import numpy as np

def prepare_state(circuit, qubits):
    circuit.x(0)  
    circuit.x(2)  
    circuit.x(4)  

def apply_hadamard(circuit, n):
    for qubit in range(n):
        circuit.h(qubit)

def apply_controlled_phase_shifts(circuit, n):
    for i in range(n):
        for j in range(i+1, n):
            angle = np.pi / (2 ** (j - i))
            circuit.cp(angle, j, i)

def bit_reversal(circuit, n):
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

def qft_5_qubits():
    n_qubits = 5
    qc = QuantumCircuit(n_qubits, n_qubits)

    prepare_state(qc, n_qubits)

    apply_hadamard(qc, n_qubits)

    apply_controlled_phase_shifts(qc, n_qubits)

    bit_reversal(qc, n_qubits)

    qc.measure(range(n_qubits), range(n_qubits))

    return qc

def run_qft():
    qc_qft = qft_5_qubits()

    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc_qft, shots=1024)
    result = job.result()
    counts = result.get_counts(qc_qft)

    print("QFT Circuit:")
    print(qc_qft.draw(output='text'))
    print("\nQFT Output:")
    print(counts)

    plot_histogram(counts).show()

def inverse_qft_5_qubits():
    n_qubits = 5
    qc = QuantumCircuit(n_qubits, n_qubits)

    prepare_state(qc, n_qubits)

    bit_reversal(qc, n_qubits)
    apply_controlled_phase_shifts(qc, n_qubits)
    apply_hadamard(qc, n_qubits)

    qc.measure(range(n_qubits), range(n_qubits))

    return qc

def run_inverse_qft():
    qc_inv_qft = inverse_qft_5_qubits()

    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc_inv_qft, shots=1024)
    result = job.result()
    counts = result.get_counts(qc_inv_qft)

    print("Inverse QFT Circuit:")
    print(qc_inv_qft.draw(output='text'))
    print("\nInverse QFT Output:")
    print(counts)

    plot_histogram(counts).show()

run_qft()

run_inverse_qft()


QFT Circuit:
     ┌───┐┌───┐                                                                »
q_0: ┤ X ├┤ H ├─■────────■────────■─────────────────■──────────────────────────»
     ├───┤└───┘ │P(π/2)  │        │                 │                          »
q_1: ┤ H ├──────■────────┼────────┼────────■────────┼─────────■────────■───────»
     ├───┤┌───┐          │P(π/4)  │        │P(π/2)  │         │        │       »
q_2: ┤ X ├┤ H ├──────────■────────┼────────■────────┼─────────┼────────┼───────»
     ├───┤└───┘                   │P(π/8)           │         │P(π/4)  │       »
q_3: ┤ H ├────────────────────────■─────────────────┼─────────■────────┼───────»
     ├───┤┌───┐                                     │P(π/16)           │P(π/8) »
q_4: ┤ X ├┤ H ├─────────────────────────────────────■──────────────────■───────»
     └───┘└───┘                                                                »
c: 5/══════════════════════════════════════════════════════════════════════════»
               

/tmp/ipykernel_87476/87011151.py:54: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot_histogram(counts).show()


Inverse QFT Circuit:
     ┌───┐                                                      ┌───┐  »
q_0: ┤ X ├────X──■────────■────────■─────────────────■──────────┤ H ├──»
     └───┘    │  │P(π/2)  │        │                 │          └───┘  »
q_1: ──────X──┼──■────────┼────────┼────────■────────┼─────────■───────»
     ┌───┐ │  │           │P(π/4)  │        │P(π/2)  │         │       »
q_2: ┤ X ├─┼──┼───────────■────────┼────────■────────┼─────────┼───────»
     └───┘ │  │                    │P(π/8)           │         │P(π/4) »
q_3: ──────X──┼────────────────────■─────────────────┼─────────■───────»
     ┌───┐    │                                      │P(π/16)          »
q_4: ┤ X ├────X──────────────────────────────────────■─────────────────»
     └───┘                                                             »
c: 5/══════════════════════════════════════════════════════════════════»
                                                                       »
«                             

/tmp/ipykernel_87476/87011151.py:83: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot_histogram(counts).show()
